In [25]:
# !sudo pip install h5py

The directory '/home/mohan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/mohan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#snimissingwarning.
  SNIMiss

In [1]:
import copy
import datetime
import numpy as np
import time
import pandas as pd
import pandas_datareader.data as web
from sklearn.cluster import KMeans
from sklearn import preprocessing
import plotly
import itertools
from talib.abstract import *
import os
import sys
from keras.models import model_from_json
from keras.models import model_from_yaml
import requests
import numpy as np
import pandas as pd
import h5py
from keras.models import Sequential
from keras.layers import Dense
from sklearn.externals import joblib
print("Import Function for the Neural Network Done")
from multiprocessing import Pool


# import plotly.plotly as py
# from plotly.graph_objs import *
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# import plotly.graph_objs as go
# plotly.tools.set_credentials_file(username='maiti.t', api_key='km8Kdfszic1Cu6ZAWiZJ')
# plotly.offline.init_notebook_mode(connected=True)

Using TensorFlow backend.


Import Function for the Neural Network Done


In [2]:
import pickle
filename = 'NeuralNetworkModel.pickle'


In [3]:
def get_open_normalised_prices(df, start, end):
    
    print('************************************************************************')
    df1 = pd.DataFrame(df["High"]/df["Open"])
    df1.columns=["H/O"]
    df1["L/O"] = df["Low"]/df["Open"]
    df1["C/O"] = df["Close"]/df["Open"]
 
    return df1

# create_follow_cluster_matrix

In [4]:
def create_follow_cluster_matrix(data):
    print('************************************************************************')
    data["ClusterTomorrow"] = data["Cluster"].shift(-1)
    data.dropna(inplace=True)
    data["ClusterTomorrow"] = data["ClusterTomorrow"].apply(int)
#     print("Tesssssssssssttttttttttttttt")
#     print(zip(data["Cluster"], data["ClusterTomorrow"]))
    data["ClusterMatrix"] = list(zip(data["Cluster"], data["ClusterTomorrow"]))
    cmvc = data["ClusterMatrix"].value_counts()
    clust_mat = np.zeros( (k, k) )
    for row in cmvc.iteritems():
#         print(row[1]*100.0/len(data))
        clust_mat[row[0]] = row[1]*100.0/len(data)
    print("Cluster Follow-on Matrix:")
    print(clust_mat)
    return clust_mat,cmvc

In [5]:
def get_indicators(stocks, period):
#     for i in stocks:
        print("***********************************************************************")
        stocks.columns = [s.lower() for s in stocks.columns]
        print("Adding Features: Started  ")
        features = pd.DataFrame(SMA(stocks, timeperiod=10))
        features.columns = ['sma_10']
        features['NATR']=pd.DataFrame(NATR(stocks, timeperiod=14))
        features = pd.concat([features,STOCHF(stocks, fastk_period=14, fastd_period=3)], axis=1)
        features['willr'] = pd.DataFrame(WILLR(stocks, timeperiod=14))
        features['rsi'] = pd.DataFrame(RSI(stocks, timeperiod=14))
        features['wma_10'] = pd.DataFrame(WMA(stocks,10))
        features['T3'] =pd.DataFrame(T3(stocks, timeperiod=5, vfactor=0))
        features['closePrice']=pd.DataFrame(stocks['close'].shift(-period))
        features['return_pct_change'] = ROC(stocks, timeperiod=period)
        features['return_pct_change'] = features['return_pct_change'].shift(-period)
        features['pct_change'] = features['return_pct_change'].apply(lambda x: '1' if x > 0 else '0' if x <= 0 else np.nan)
        features = features.dropna()
           
        return features

In [6]:
def modellingNeuralNetwork(trainX,trainY,numbFeatures,i):
    print("Creating Neural Network .....")
    starttime= time.time()
    model = Sequential()
    model.add(Dense(5, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(11, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(11, input_dim=numbFeatures, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'mape'])
    history1 = model.fit(trainX, trainY, epochs=20, batch_size=2, verbose=0)
    print("Time Taken to Model %s seconds ---" % (time.time() - starttime))
    print("*******************************************")
    print(history1)
    print("*******************************************")
    scores = model.evaluate(trainX,trainY,verbose = 0)
#     scores = model.evaluate(X, Y, verbose=0)
#     print("%s: %.2f%%" % (model.metrics_names[0], scores[1]*100))
    print(model.summary())
    # serialize model to JSON
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    h5filename = 'model'+str(i)+'.h5'
    model.save_weights(h5filename)
    print("Saved model to disk")
#     picklefilename = 'model'+str(i)+'.pkl'
#     joblib.dump(model, picklefilename)
#     joblib.dump(model, 'model'+str(i)+'.pkl')

    return model,history1

In [7]:
def create_dataset1(datasetReal,i):
    print(type(datasetReal))
    xdataset = datasetReal.copy()
    xdataset = xdataset[xdataset.columns[xdataset.columns !='closePrice']]
    dataset = np.array(xdataset)
    print("Total Number of features in trainig Dataset "+ str(xdataset.columns.size))
    dataX = [dataset[n] for n in range(len(dataset)-i)]
    return np.array(dataX),xdataset.columns.size

In [8]:
predictedClosePrice2 = []
frameshistory=[]
# model = []
def testingModel1(inputClusterone):
    
        starttime1= time.time()
              
        
        for i in range(1,8):
            print("Training the Neural Network for next "+ str(i)+" Prediction")
            starttime= time.time() 
            
            print("Copying the input Dataset ")
            inputdata = inputClusterone.copy()
            
            dataset1 = get_indicators(inputClusterone,i)
            if(i ==1):
                print("Getting the features for Test data ")
                testdata = dataset1.tail(1)
                joblib.dump(testdata, 'testdata2.pkl')

            print("Making The Training Dataset")
            traindataset = dataset1.head(len(dataset1)-2)

            trainY = np.array(traindataset['closePrice'])[i:]
            trainX, numbFeatures = create_dataset1(traindataset,i)
            
#             m,h = modellingNeuralNetwork(trainX,trainY,numbFeatures,i)
            
            json_file = open('model'+str(i)+'.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            loaded_model = model_from_json(loaded_model_json)
            # load weights into new model
            modelname = 'model'+str(i)+'.h5' 
            loaded_model.load_weights(modelname)
            print("Loaded model"+ str(i)+" from disk")

            # evaluate loaded model on test data
            loaded_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'mape'])
            
#             f={"models":m}
#             s = pickle.dumps( m, "model"+str(i)+".pkl" ) 
#             s = pickle.dumps(m) 
#             joblib.dump(s, 'model'+str(i)+'.pkl')
            print("Model Compiled")
#             joblib.dump(m, '"model"+str(i)+".pkl"') 
#             model.append(m)
            
#             fullReport  = pd.DataFrame(h.history)
#             fullReport.to_csv("Day_"+ str(i)+" NN_PredictionModel.csv")
#             finalMAPE=fullReport['mean_absolute_percentage_error'].tail(1)
#             finalLoss = fullReport['loss'].tail(1)
#             finalMAE = fullReport['mean_absolute_error'].tail(1)

#             finalreport = pd.DataFrame(finalMAPE)
#             finalreport.columns = ['MAPE']
#             finalreport['Loss']=finalLoss
#             finalreport['MAE']=finalMAE
#             finalreport.index = range(1,len(finalreport)+1)

#             frameshistory.append(finalreport)
#             print(finalreport)

            print("Testing on the Latest Trade Data ...")

#             trainfeature=traindataset.tail(len(traindataset)-1)
#             trainlabel= trainfeature['closePrice']

#             prediction = m.predict(trainX)
#             my_list = map(lambda x: x[0], prediction)
#             traintestlabel = pd.Series(my_list)

#             testdata = dataset1.tail(1)
            testdata = joblib.load('testdata2.pkl')
#             testdata = testDataLatestTrade
#             testexpectedlabel = testdata['closePrice']

            xtest = testdata[testdata.columns[testdata.columns !='closePrice']]
            print("-----------------------------------------")
            print("Test record Given")
            print(xtest)
            print("----------------------------------")
            
            predictiontest = loaded_model.predict(np.array(xtest))
            
            
            temp_list = map(lambda x: x[0], predictiontest)
            testlabel = pd.Series(temp_list)
            print("**********************************************************")
#             print("Excpected Close Price "+ str( testexpectedlabel.values))
            print("Predicted Close Price  "+ str(testlabel[0]))
#             predictedClosePrice.append(testlabel[0])
            clf = joblib.load('predictedClosePrice2.pkl') 
            clf.append(testlabel[0])
            joblib.dump(clf, 'predictedClosePrice2.pkl')
            print("**********************************************************")
            print("Time Taken to Model %s seconds ---" % (time.time() - starttime))
            
        print("Testing the Model Finished ...")
        print("Total Time Taken to Model %s seconds ---" % (time.time() - starttime1))
#         allrecord = pd.concat(frameshistory)
#         allrecord.index = range(1,len(allrecord)+1)
#         print("Final Model Evaluation Scores ")
#         print(allrecord)


In [9]:
def datadownload(start,end):
    try:
        dow = web.DataReader("^DJI", "yahoo", start, end)
        dow.to_csv("AllDJIdata.csv")
    except (RuntimeError, TypeError, NameError,Exception):
                   print("Oops! Yahoo Data Service is not working ....")
                   print("Data Taken from last updated repository")
                   dow=pd.read_csv("AllDJIdata.csv")
        
    
    return dow
    

In [10]:
def knn(dow_norm,k):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(dow_norm)
    labels = km.labels_
    return labels,km

In [11]:
def getinformationCluster(cmvc,km):
    print("Ranking of Clusters ");
    print(str(cmvc))
    print('************************************************************************')
    print("Centroid for the K Means Clusters ")
    kmean = pd.DataFrame(km.cluster_centers_)
    kmean.columns=['X','Y','Z']
    print(kmean)
    print('************************************************************************')
    

In [12]:
def predictCluster(km,dow_norm,cmvc,clust_mat):
    print("Getting the Latest Trade OHLC Values for DOW 30 ") 
    testdoeN = dow_norm.copy()
    print("*****************************************8")
#     print(testdoeN.tail(5))
#     del testdoeN['Cluster']
#     print("Testtttttt")
    
    print(testdoeN.tail(1).values.reshape(-1,3))
    predictedCluster = km.predict(testdoeN.tail(1).values.reshape(-1,3))[0]
    print("Predicted Cluster of Last Trade OHLC Date "+ " "+ str(testdoeN.index[len(testdoeN)-1])+" "+ str(predictedCluster))
    predictedNextCluster = []
    for i,row in cmvc.iteritems():
        if(i[0]==predictedCluster):
            predictedNextCluster.append(i[1])
    print("Predicted Next Day Cluster would be : "+ str(predictedNextCluster))
    
    probabCluster=[]
    totalPrbablityfromPast=0
    for i in predictedNextCluster:
            totalPrbablityfromPast=totalPrbablityfromPast+clust_mat[predictedCluster][i]
    #         probabCluster.append(clust_mat[predictedCluster][i])
    print("Calulating the Relative Probablity ")
    for i in predictedNextCluster:
            probabCluster.append(clust_mat[predictedCluster][i]/totalPrbablityfromPast*100)
            
    nextClusterMatrix = pd.DataFrame(predictedNextCluster)
    nextClusterMatrix.columns=["Cluster"]
    nextClusterMatrix['Probability']=probabCluster
    
    print("Next Day CLuster and its Relative probablity from the Predicted Cluster : "+ str(predictedCluster))
    print(nextClusterMatrix)
    return nextClusterMatrix
    


In [13]:
def getRightClusteredData(nextClusterMatrix,dowtrain):
    print("Getting the filtered Past data where the maximum predicted range of the next DAY OHLC would be.. ")
    inputClusterone = dowtrain[dowtrain['Cluster']==nextClusterMatrix['Cluster'][0]]
    return inputClusterone
    

In [14]:
# import pandas_datareader.data as web
# start = datetime.datetime(2005, 1, 1)
# end =  time.strftime("%Y-%m-%d")
# dow = web.DataReader("^DJI", "yahoo", start, end)


In [15]:
# dow=pd.read_csv("AllDJIdata.csv")

In [16]:
# frameshistory=[]
# def testingModel(i):
    
            
              
        
# #         for i in range(1,3):
#             print("Training the Neural Network for next "+ str(i)+" Prediction")
#             starttime= time.time() 
            
#             print("Copying the input Dataset ")
#             inputdata = inputClusterone.copy()
            
#             dataset1 = get_indicators(inputClusterone,i)

#             print("Making The Training Dataset")
#             traindataset = dataset1.head(len(dataset1)-2)

#             trainY = np.array(traindataset['closePrice'])[i:]
#             trainX, numbFeatures = create_dataset1(traindataset,i)
            
#             m,h = modellingNeuralNetwork(trainX,trainY,numbFeatures,i)
            
# #             f={"models":m}
# #             s = pickle.dumps( m, "model"+str(i)+".pkl" ) 
# #             s = pickle.dumps(m) 
            
#             print("Model Saved")
# #             joblib.dump(m, '"model"+str(i)+".pkl"') 
# #             model.append(m)
            
#             fullReport  = pd.DataFrame(h.history)
# #             fullReport.to_csv("Day_"+ str(i)+" NN_PredictionModel.csv")
#             finalMAPE=fullReport['mean_absolute_percentage_error'].tail(1)
#             finalLoss = fullReport['loss'].tail(1)
#             finalMAE = fullReport['mean_absolute_error'].tail(1)

#             finalreport = pd.DataFrame(finalMAPE)
#             finalreport.columns = ['MAPE']
#             finalreport['Loss']=finalLoss
#             finalreport['MAE']=finalMAE
#             finalreport.index = range(1,len(finalreport)+1)

#             frameshistory.append(finalreport)
#             print(finalreport)

#             print("Testing on the Latest Trade Data ...")

# #             trainfeature=traindataset.tail(len(traindataset)-1)
# #             trainlabel= trainfeature['closePrice']

# #             prediction = m.predict(trainX)
# #             my_list = map(lambda x: x[0], prediction)
# #             traintestlabel = pd.Series(my_list)

#             testdata = dataset1.tail(1)
# #             testdata = testDataLatestTrade
# #             testexpectedlabel = testdata['closePrice']

#             xtest = testdata[testdata.columns[testdata.columns !='closePrice']]
#             predictiontest = m.predict(np.array(xtest))
            
            
#             temp_list = map(lambda x: x[0], predictiontest)
#             testlabel = pd.Series(temp_list)
#             print("**********************************************************")
# #             print("Excpected Close Price "+ str( testexpectedlabel.values))
#             print("Predicted Close Price  "+ str(testlabel[0]))
    
#             clf = joblib.load('predictedClosePrice2.pkl') 
#             clf.append(testlabel[0])
#             joblib.dump(clf, 'predictedClosePrice2.pkl')
# #             predictedClosePrice.append(testlabel[0])
#             print("**********************************************************")
#             print("Time Taken to Model %s seconds ---" % (time.time() - starttime))
            


In [17]:
if __name__ == "__main__":
    # Obtain DOW 30  pricing data from Yahoo Finance
    print('************************************************************************')
    print("Step 1:Downloading the DOW 30 data from Yahoo Finance ")
    start = datetime.datetime(2005, 1, 1)
    end =  time.strftime("%Y-%m-%d")
#     dow = web.DataReader("^DJI", "yahoo", start, end)
    dow = datadownload(start,end)
    print("Status ::Data Download Finish....")
    print('************************************************************************')
    print("Step 2:Normalising the Data with respect to Daily Open Stock Price ")
    
    dow_norm = get_open_normalised_prices(dow, start, end)
    print("Number of Chosen Cluster : 5 ")
    
    k = 5
    print('************************************************************************')
    print("Step 3: KNN Process Started ...")
    
    # Deleting the last row from the KNN Dataset 
    traindowNorm = dow_norm.copy()
    traindowNorm.drop(traindowNorm.index[len(traindowNorm)-1])
    
    # Reatining the latest Trade info as normalised one
    testDataLatestTrade = dow_norm.tail(1)
    
    labels,km = knn(traindowNorm,k)
    
    dowtrain = dow.copy()
    dowtrain.drop(dowtrain.index[len(dowtrain)-1])
    dowtrain["Cluster"] = labels
    traindowNorm["Cluster"] = labels
    
    print("Status : KNN Process Finished ...")
    # Create and output the cluster follow-on matrix
    clust_mat,cmvc = create_follow_cluster_matrix(dowtrain)
    
    print('************************************************************************')
    
    getinformationCluster(cmvc,km)
    
    nextClusterMatrix = predictCluster(km,testDataLatestTrade,cmvc,clust_mat)
    
    inputClusterone = getRightClusteredData(nextClusterMatrix,dowtrain)
    
    inputClusterone.append(dow.tail(1))
    
    joblib.dump(predictedClosePrice2, 'predictedClosePrice2.pkl')
#     p = Pool(20)
# #     frameshistory=[]
#     print(p.map(testingModel, range (1,4)))
    testingModel1(inputClusterone)
#     print("Testing the Model Finished ...")
#     allrecord = pd.concat(frameshistory)
#     allrecord.index = range(1,len(allrecord)+1)
#     print("Final Model Evaluation Scores ")
#     print(allrecord)
    # load json and create model
    clf = joblib.load('predictedClosePrice2.pkl')
    print("**************************************")
    print(clf)
    



************************************************************************
Step 1:Downloading the DOW 30 data from Yahoo Finance 
Status ::Data Download Finish....
************************************************************************
Step 2:Normalising the Data with respect to Daily Open Stock Price 
************************************************************************
Number of Chosen Cluster : 5 
************************************************************************
Step 3: KNN Process Started ...
Status : KNN Process Finished ...
************************************************************************
Cluster Follow-on Matrix:
[[  5.63070148   4.62409563  14.21830764   0.2831079    0.66058509]
 [  5.37905002   2.67379679   5.94526581   0.44039006   0.72349796]
 [ 13.33752752   6.85750236  33.9100346    0.2831079    0.78641082]
 [  0.34602076   0.34602076   0.12582573   0.15728216   0.44039006]
 [  0.72349796   0.66058509   0.97514942   0.25165146   0.22019503]]
***************

Making The Training Dataset
<class 'pandas.core.frame.DataFrame'>
Total Number of features in trainig Dataset 10
Loaded model6 from disk
Model Compiled
Testing on the Latest Trade Data ...
-----------------------------------------
Test record Given
                  sma_10      NATR      fastk      fastd      willr       rsi  \
Date                                                                            
2017-08-15  22008.264063  0.474287  63.335255  57.544256 -36.664745  62.57257   

                  wma_10            T3  return_pct_change pct_change  
Date                                                                  
2017-08-15  21986.315945  21966.189002           0.117637          1  
----------------------------------
**********************************************************
Predicted Close Price  22197.3144531
**********************************************************
Time Taken to Model 0.148769855499 seconds ---
Training the Neural Network for next 7 Prediction
Copying

In [18]:
clf = joblib.load('predictedClosePrice2.pkl')

In [19]:
clf 

[22474.2265625,
 22027.138671875,
 22208.052734375,
 21904.787109375,
 22384.896484375,
 22197.314453125,
 22250.8203125]